<a href="https://colab.research.google.com/github/dmipatriot/fantasy_football_reports/blob/main/ESPN_Fantasy_Roast_SLIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Config & Imports ===
!pip install espn-api pandas numpy requests python-dotenv

import os, json, re, math
from datetime import datetime, timezone
from pathlib import Path
from google.colab import userdata

import pandas as pd
import numpy as np


from espn_api.football import League

try:
    import requests
except Exception:
    requests = None  # posting optional

import os

# ---- Env / Defaults ----

LEAGUE_ID = userdata.get("LEAGUE_ID")
ESPN_S2 = userdata.get('ESPN_S2')
SWID = userdata.get('SWID')


#LEAGUE_ID = int(os.getenv("FW_LEAGUE_ID", "577848"))              # REQUIRED for private leagues
SEASON    = int(os.getenv("FW_SEASON", "2025"))
WEEK      = int(os.getenv("FW_WEEK", "10"))

#ESPN_S2   = os.getenv("FW_ESPN_S2") or os.getenv("ESPN_S2")
#SWID      = os.getenv("FW_SWID")

OUTPUT_DIR = Path(os.getenv("FW_OUTPUT_DIR", "fantasy_roast_output"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Slim-only export
WEBHOOK_URL = userdata.get('WEBHOOK_URL_ROAST')
POST_TO_WEBHOOK = True

# Gate heavy extras; default off
INCLUDE_EXTRAS = (os.getenv("FW_INCLUDE_EXTRAS", "false").lower() == "true")

SCHEMA_VERSION = "2.1.0"

def now_iso():
    return datetime.now(timezone.utc).isoformat()


print(f"[Cell 1 OK] Config & imports loaded. Season={SEASON}, Week={WEEK}, League_ID={LEAGUE_ID}")


In [ ]:
# === Helpers ===
def qround(x, ndigits=2):
    """Quiet round: returns None for None/NaN, else rounded float."""
    if x is None:
        return None
    try:
        if isinstance(x, (int, float, np.floating)) and not math.isnan(float(x)):
            return round(float(x), ndigits)
    except Exception:
        pass
    return None

def canon_team_name(name: str) -> str:
    if not name:
        return ""
    return re.sub(r"\s+", " ", str(name).strip())

def safe_get_projection(team_or_player):
    """Try known attributes for projection; return None when not present."""
    for attr in ("projected_total", "projected_points", "projected"):
        if hasattr(team_or_player, attr):
            val = getattr(team_or_player, attr)
            try:
                return float(val) if val is not None else None
            except Exception:
                continue
    return None

def team_name_from_obj(team):
    # espn-api usually has team.team_name; fallback to location/nickname/owner
    for attr in ("team_name", "team_nickname", "team_abbrev", "location", "nickname"):
        if getattr(team, attr, None):
            return canon_team_name(getattr(team, attr))
    owner = getattr(team, "owners", None) or getattr(team, "owner", None)
    return canon_team_name(str(owner) if owner else f"Team {getattr(team, 'team_id', '?')}")

def is_starter(slot_position: str) -> bool:
    """Exclude bench/IL; espn-api slot names commonly 'BE' for bench."""
    if not slot_position:
        return True
    sp = str(slot_position).upper()
    return sp not in {"BE", "BENCH", "IR", "IL", "IR-R", "OUT"}

def sum_projected_from_lineup(lineup, starters_only=True):
    total = 0.0
    has_any = False
    for p in lineup:
        if starters_only and not is_starter(getattr(p, "slot_position", None)):
            continue
        proj = safe_get_projection(p)  # uses projected_points, projected_total, etc.
        if proj is not None:
            total += float(proj)
            has_any = True
    return qround(total) if has_any else None




print("[Cell 2 OK] Helper functions ready (qround, canon_team_name, safe_get_projection, etc.)")

In [ ]:
# === League init ===
if LEAGUE_ID == 0:
    raise RuntimeError("FW_LEAGUE_ID must be provided (int).")

league = League(
    league_id=LEAGUE_ID,
    year=SEASON,
    espn_s2=ESPN_S2,
    swid=SWID
)

# League name with robust fallback
league_name = getattr(league, "league_name", None) \
          or getattr(getattr(league, "settings", None), "name", None) \
          or f"League {LEAGUE_ID}"
league_name = canon_team_name(league_name)


print(f"[Cell 3 OK] League initialized: {league_name} (ID={LEAGUE_ID}, Season={SEASON})")


In [ ]:
# === Extraction ===
def extract_matchups(league, week: int):
    ms = []
    for bs in league.box_scores(week=week):
        home = team_name_from_obj(bs.home_team)
        away = team_name_from_obj(bs.away_team)
        home_score = qround(bs.home_score)
        away_score = qround(bs.away_score)
        margin = qround((home_score or 0) - (away_score or 0))
        ms.append({
            "home_team": home,
            "away_team": away,
            "home_score": home_score,
            "away_score": away_score,
            "margin": margin
        })
    return ms

def extract_players_from_boxscores(league, week: int):
    per_team_players = {}
    for bs in league.box_scores(week=week):
        for side, team, roster in (
            ("home", bs.home_team, bs.home_lineup),
            ("away", bs.away_team, bs.away_lineup)
        ):
            tname = team_name_from_obj(team)
            players = per_team_players.setdefault(tname, [])
            for p in roster:
                players.append({
                    "player": canon_team_name(getattr(p, "name", "")),
                    "proTeam": getattr(p, "proTeam", None),
                    "position": getattr(p, "position", None),
                    "slot_position": getattr(p, "slot_position", None),
                    "points": qround(getattr(p, "points", None)),
                    "projected_points": qround(safe_get_projection(p)),
                    "injuryStatus": getattr(p, "injuryStatus", None),
                })
    return per_team_players

def extract_team_records(league):
    records = {}
    for t in league.teams:
        name = team_name_from_obj(t)
        records[name] = {
            "wins": int(getattr(t, "wins", 0) or 0),
            "losses": int(getattr(t, "losses", 0) or 0),
            "ties": int(getattr(t, "ties", 0) or 0),
            "points_for": qround(getattr(t, "points_for", None)),
            "points_against": qround(getattr(t, "points_against", None)),
        }
    return records


def extract_matchups(league, week: int):
    ms = []
    for bs in league.box_scores(week=week):
        home = team_name_from_obj(bs.home_team)
        away = team_name_from_obj(bs.away_team)

        home_score = qround(bs.home_score)
        away_score = qround(bs.away_score)
        margin = qround((home_score or 0) - (away_score or 0))

        # NEW: projecteds from the lineups (starters only)
        home_proj = sum_projected_from_lineup(bs.home_lineup, starters_only=True)
        away_proj = sum_projected_from_lineup(bs.away_lineup, starters_only=True)

        ms.append({
            "home_team": home,
            "away_team": away,
            "home_score": home_score,
            "away_score": away_score,
            "home_projected": home_proj,     # <-- added
            "away_projected": away_proj,     # <-- added
            "margin": margin
        })
    return ms



print("[Cell 4 OK] Extraction functions defined (matchups, players, records)")



In [ ]:
# === Computations ===
def build_indexed_matchups(matchups):
    """Return maps to quickly find opponent/result/margin for each team."""
    opp = {}
    res = {}
    mar = {}
    pts = {}
    for m in matchups:
        h, a = m["home_team"], m["away_team"]
        hs, as_ = m["home_score"] or 0, m["away_score"] or 0
        pts[h] = hs; pts[a] = as_
        opp[h] = a; opp[a] = h
        margin = qround(hs - as_)
        mar[h] = margin
        mar[a] = qround(-margin if margin is not None else None)
        if hs > as_:
            res[h] = "W"; res[a] = "L"
        elif hs < as_:
            res[h] = "L"; res[a] = "W"
        else:
            res[h] = res[a] = "T"
    return opp, res, mar, pts

def compute_projections(league, week):
    """Return simple dict of team->weekly projection (None if missing)."""
    proj = {}
    for t in league.teams:
        name = team_name_from_obj(t)
        proj[name] = qround(safe_get_projection(t))
    return proj

def compute_deltas(actual_pts, projected_pts):
    deltas = []
    for team, actual in actual_pts.items():
        proj = projected_pts.get(team)
        d = (actual - proj) if (actual is not None and proj is not None) else None
        deltas.append({
            "team": team,
            "actual": qround(actual),
            "projected": qround(proj),
            "delta": qround(d) if d is not None else None
        })
    return deltas

def compute_awards(matchups, per_team_players, deltas):
    # MVP: highest single player points
    mvp_player = None
    for team, players in per_team_players.items():
        for p in players:
            pts = p.get("points")
            if pts is None:
                continue
            if (mvp_player is None) or (pts > mvp_player["points"]):
                mvp_player = {"player": p["player"], "team": team, "points": pts}

    # Blowout: largest absolute margin among matchups
    blowout = None
    for m in matchups:
        abs_margin = abs(m["margin"] or 0)
        if (blowout is None) or (abs_margin > abs(blowout["margin"] or 0)):
            # Winner is who has positive margin
            if (m["margin"] or 0) > 0:
                winner = m["home_team"]
            elif (m["margin"] or 0) < 0:
                winner = m["away_team"]
            else:
                winner = m["home_team"]  # tie; arbitrary
            blowout = {"team": winner, "margin": abs_margin}

    # Luckiest/Unluckiest by delta (actual - projected)
    # (ignore teams with missing projections)
    with_d = [d for d in deltas if d.get("delta") is not None]
    luckiest = max(with_d, key=lambda x: x["delta"]) if with_d else None
    unluckiest = min(with_d, key=lambda x: x["delta"]) if with_d else None

    return {
        "mvp": mvp_player,
        "blowout": blowout,
        "luckiest": luckiest,
        "unluckiest": unluckiest
    }


def compute_weekly_team_projections(league, week):
    """Dict: team -> projected total for THIS week (starters-only sum)."""
    proj = {}
    for bs in league.box_scores(week=week):
        home = team_name_from_obj(bs.home_team)
        away = team_name_from_obj(bs.away_team)
        proj[home] = sum_projected_from_lineup(bs.home_lineup, starters_only=True)
        proj[away] = sum_projected_from_lineup(bs.away_lineup, starters_only=True)
    return proj


print("[Cell 5 OK] Computation functions defined (indices, deltas, awards)")


In [ ]:
# === Build payload v2.1 ===
def build_payload_v21(league, week, include_extras=False):
    matchups = extract_matchups(league, week)
    per_team_players = extract_players_from_boxscores(league, week)
    team_records = extract_team_records(league)
    projections = compute_weekly_team_projections(league, week)

    # Indices for opponent/result/margin and actual weekly points
    opp, res, mar, actual_pts = build_indexed_matchups(matchups)
    deltas = compute_deltas(actual_pts, projections)
    awards = compute_awards(matchups, per_team_players, deltas)

    # per_team block (starters-only top 3 by points)
    per_team = []
    for team in sorted(team_records.keys()):
        players = per_team_players.get(team, [])
        starters = [p for p in players if is_starter(p.get("slot_position"))]
        starters_sorted = sorted(
            [p for p in starters if p.get("points") is not None],
            key=lambda x: x["points"],
            reverse=True
        )
        top_players = [
            {"player": p["player"], "points": p["points"], "position": p.get("position")}
            for p in starters_sorted[:3]
        ]

        per_team.append({
            "team": team,
            "record": team_records[team],
            "week_points": qround(actual_pts.get(team)),
            "week_projected": qround(projections.get(team)),
            "opponent": opp.get(team),
            "result": res.get(team),
            "margin": mar.get(team),
            "top_players": top_players
        })

    payload = {
        "meta": {
            "source": "espn-api",
            "schema_version": SCHEMA_VERSION,
            "generated_at": now_iso(),
            "season": SEASON,
            "week": WEEK,
            "league_id": getattr(league, "league_id", None),
            "league_name": league_name
        },
        "awards": awards,
        "matchups": matchups,
        "per_team": per_team,
        "deltas": deltas
    }

    if include_extras:
        payload["extras"] = {"all_play_week": []}

    return payload

print("[Cell 6 OK] Payload builder v2.1 ready")


In [ ]:
# === Run & Write (Cell #7) ===
payload_v21 = build_payload_v21(league, WEEK, include_extras=INCLUDE_EXTRAS)

# Debug: verify projections exist
_projections_dbg = compute_weekly_team_projections(league, WEEK)
_missing_proj = sorted([t for t, v in _projections_dbg.items() if v is None])
print(f"[Debug] Teams with missing weekly projections: {_missing_proj or 'None'}")

# Show a few matchup projections
for m in payload_v21["matchups"][:3]:
    print(f"[Debug] {m['home_team']} proj={m.get('home_projected')} "
          f"vs {m['away_team']} proj={m.get('away_projected')}")

# Spot-check per_team projections
print("[Check] Example per_team rows with projections:")
for row in payload_v21["per_team"][:3]:
    print("   ", row["team"], "→ week_points:", row["week_points"],
          "week_projected:", row["week_projected"])

# Write JSON payload
json_path = OUTPUT_DIR / f"fantasy_roast_week_{WEEK}_v21.json"
with json_path.open("w", encoding="utf-8") as f:
    json.dump(payload_v21, f, ensure_ascii=False, indent=2)

# Define and write minimal schema for v2.1
schema = {
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "FantasyRoast v2.1",
  "type": "object",
  "required": ["meta", "awards", "matchups", "per_team", "deltas"],
  "properties": {
    "meta": {
      "type": "object",
      "required": ["schema_version","season","week","league_id","league_name","generated_at"],
      "properties": {
        "source": {"type": "string"},
        "schema_version": {"const": "2.1.0"},
        "generated_at": {"type": "string"},
        "season": {"type": "integer"},
        "week": {"type": "integer"},
        "league_id": {"type": ["integer", "string"]},
        "league_name": {"type": "string"}
      }
    },
    "awards": {"type":"object"},
    "matchups": {"type":"array"},
    "per_team": {"type":"array"},
    "deltas": {"type":"array"},
    "extras": {"type":"object"}
  }
}
schema_path = OUTPUT_DIR / "FantasyRoast_JSON_Schema_v2_1.json"
with schema_path.open("w", encoding="utf-8") as f:
    json.dump(schema, f, ensure_ascii=False, indent=2)

print(f"[Cell 7 OK] Wrote slim payload to {json_path} and schema to {schema_path}")

# ---------- POST TO MAKE WEBHOOK ----------
# Requires: POST_TO_WEBHOOK=True and WEBHOOK_URL set in Cell 1, and `requests` available
if POST_TO_WEBHOOK and WEBHOOK_URL and requests:
    try:
        print("Posting payload to:", WEBHOOK_URL)
        r = requests.post(WEBHOOK_URL, json=payload_v21, timeout=20)
        print("Webhook POST status:", r.status_code)
        # Optional: show short body on non-2xx
        if r.status_code // 100 != 2:
            print("Webhook response snippet:", r.text[:300])
    except Exception as e:
        print("Webhook POST failed:", e)
else:
    print("Skipping POST (set POST_TO_WEBHOOK=True and FW_WEBHOOK_URL to enable).")


In [ ]:
# === JSON Validation (Cell #8) ===
# Validates the v2.1 payload against the schema. Fails fast with readable errors.

import json, sys
from pathlib import Path

# Try to import jsonschema; install if needed
try:
    import jsonschema
except Exception:
    import subprocess
    print("Installing jsonschema…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "jsonschema>=4.22.0"])
    import jsonschema

# Resolve paths written in Cell #7
json_path = OUTPUT_DIR / f"fantasy_roast_week_{WEEK}_v21.json"
schema_path = OUTPUT_DIR / "FantasyRoast_JSON_Schema_v2_1.json"

# Load payload (prefer in-memory variable from prior cells, else from disk)
try:
    _payload = payload_v21  # noqa: F821
except NameError:
    with json_path.open("r", encoding="utf-8") as f:
        _payload = json.load(f)

# Load schema (prefer in-memory dict from prior cells, else from disk)
try:
    _schema = schema  # noqa: F821
except NameError:
    with schema_path.open("r", encoding="utf-8") as f:
        _schema = json.load(f)

# Helper to make error paths readable
def path_str(err):
    parts = ["$"]
    for p in list(err.absolute_path):
        if isinstance(p, int):
            parts.append(f"[{p}]")
        else:
            parts.append(f".{p}")
    return "".join(parts)

validator = jsonschema.Draft202012Validator(_schema)
errors = sorted(validator.iter_errors(_payload), key=lambda e: e.path)

if errors:
    print(f"❌ Schema validation FAILED with {len(errors)} error(s):")
    for i, e in enumerate(errors, 1):
        print(f"  {i:02d}. {path_str(e)} — {e.message}")
    raise RuntimeError("Payload does not conform to FantasyRoast v2.1 schema.")
else:
    print(f"✅ Schema validation PASSED for {json_path.name}")
    # Optional: additional sanity checks beyond the schema (keep fast & light)
    # 1) Ensure 'survivor' key is not present
    if "survivor" in _payload:
        raise RuntimeError("Found unexpected key: 'survivor' (should be removed in v2.1).")
    # 2) Ensure team names are non-empty strings
    bad_names = [t for t in _payload.get("per_team", []) if not t.get("team")]
    if bad_names:
        raise RuntimeError("One or more per_team entries have missing/empty 'team' names.")
    print("🔎 Quick sanity checks passed.")

